## Autoencoder with Convolutional Neural Networks

Simple tutorial to train an [autoencoder](https://en.wikipedia.org/wiki/Autoencoder) with convolutional neural networks in Torch.

As a toy dataset, we will use the common MNIST dataset. More information about MNIST can be found at http://yann.lecun.com/exdb/mnist/.

First, we import some dependecies.

In [1]:
require 'torch'
require 'nn'
require 'optim'
mnist = require 'mnist'

We load and shuffle the data.

In [2]:
train_data = mnist.traindataset()
test_data = mnist.testdataset()

perm = torch.randperm(train_data.label:size(1)):long()
train_data.data = train_data.data:index(1, perm):float()
train_data.label = train_data.label:index(1, perm)

test_data.data = test_data.data:float()

We print one image as an example.

In [3]:
itorch.image(train_data.data[9])

We create the network and print the network's architecture.

In [4]:
function create_model()
    net = nn.Sequential() -- input (1, 28, 28)
    
    -- encoder
    net:add(nn.Reshape(1, 28, 28)) -- (1, 28, 28)
    net:add(nn.SpatialConvolution(1, 8, 5, 5)) -- (8, 24, 24)
    net:add(nn.ReLU())
    net:add(nn.SpatialConvolution(8, 16, 5, 5)) --  (16, 20, 20)
    net:add(nn.ReLU())
    net:add(nn.SpatialMaxPooling(2, 2, 2, 2)) -- (16, 10, 10)
    net:add(nn.Reshape(16*10*10))

    -- code / low-dimensional representation
    net:add(nn.Linear(16*10*10, 128))
    net:add(nn.ReLU())
    
    -- decoder
    net:add(nn.Linear(128, 16*10*10))
    net:add(nn.ReLU())
    net:add(nn.Reshape(16, 10, 10)) -- (16, 10, 10)
    net:add(nn.SpatialUpSamplingNearest(2)) -- (16, 20, 20)
    net:add(nn.SpatialZeroPadding(4, 4, 4, 4)) -- (16, 28, 28)
    net:add(nn.SpatialConvolution(16, 8, 5, 5)) -- (8, 24, 24)
    net:add(nn.ReLU())
    net:add(nn.SpatialZeroPadding(4, 4, 4, 4)) -- (8, 32, 32)
    net:add(nn.SpatialConvolution(8, 1, 5, 5)) -- (1, 28, 28)
    return net
end

In [5]:
model = create_model()
print(model:__tostring())

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> (17) -> (18) -> output]
  (1): nn.Reshape(1x28x28)
  (2): nn.SpatialConvolution(1 -> 8, 5x5)
  (3): nn.ReLU
  (4): nn.SpatialConvolution(8 -> 16, 5x5)
  (5): nn.ReLU
  (6): nn.SpatialMaxPooling(2x2, 2,2)
  (7): nn.Reshape(1600)
  (8): nn.Linear(1600 -> 128)
  (9): nn.ReLU
  (10): nn.Linear(128 -> 1600)
  (11): nn.ReLU
  (12): nn.Reshape(16x10x10)
  (13): nn.SpatialUpSamplingNearest
  (14): nn.SpatialZeroPadding(l=4, r=4, t=4, b=4)
  (15): nn.SpatialConvolution(16 -> 8, 5x5)
  (16): nn.ReLU
  (17): nn.SpatialZeroPadding(l=4, r=4, t=4, b=4)
  (18): nn.SpatialConvolution(8 -> 1, 5x5)
}	


We create the loss function. We use the *mean squared error* (MSE) to measure the difference between the input and its reconstruction. We aim at minimizing the MSE.

In [6]:
criterion = nn.MSECriterion():float()

Defining now hyperparameters related to the network training.

In [7]:
optimState = {learningRate=0.01, momentum=0.5} -- parameters for the (minibatch) gradient descent algorithm

batchSize = 128
maxEpoch = 20 -- train for 20 epochs

display_n = 500000 -- display training loss and accuracy every x step
test_n = 500 -- test the model every x step

In [8]:
-- placeholder for batch
X_batch = train_data.data[{{1,batchSize}}]:clone():float()

-- placeholder for batch indices
batchIndices = torch.LongTensor(batchSize)

Optionally, we can cast the model created as well as the different variables on GPU.

In [9]:
-- put on GPU
require 'cutorch'
require 'cunn'
cutorch.setDevice(2) -- chose which GPU to use
model = model:cuda()
criterion = criterion:cuda()
train_data.data = train_data.data:cuda()
test_data.data = test_data.data:cuda()
X_batch = X_batch:cuda()

We now train the network.

In [12]:
-- model parameters for optimization
local params, gradParams = model:getParameters()
local iteration = 0

local steps_per_epoch = train_data.label:size(1) / batchSize

while true do
   iteration = iteration + 1
    
   batchIndices:random(1, train_data.label:size(1))
   X_batch:copy(train_data.data:index(1, batchIndices))

    ----------------------------------------
    -------------- TRAINING ----------------
    ----------------------------------------
   function feval(params)
      gradParams:zero()
      outputs = model:forward(X_batch)
      loss = criterion:forward(outputs, X_batch)
      local gradOutputs = criterion:backward(outputs, X_batch)
      model:backward(X_batch, gradOutputs)
      return loss, gradParams
   end
    
   timer = torch.Timer() -- timer
   optim.adagrad(feval, params, optimState)
   datum_sec = batchSize / 
   timer:time().real
    
   if iteration % display_n == 0 then
          print(string.format("TRAINING – epoch %.2f, loss = %.4f, %.2f datum/sec", iteration / steps_per_epoch, loss, datum_sec))
   end
    ----------------------------------------
    ----------------------------------------
    
    
    ----------------------------------------
    ------------- EVALUATION ---------------
    ----------------------------------------
   if (iteration - 1) % test_n == 0 or iteration / steps_per_epoch >= maxEpoch then
       model:evaluate() -- evaluation mode, i.e. do not use dropout
       local outputs_ = model:forward(test_data.data)
       local loss_ = criterion:forward(outputs, test_data.data)
       print(string.format('TEST – epoch %.2f, loss: %.4f', iteration / steps_per_epoch, loss_))
       model:training() -- training mode, i.e. use dropout
   end
    ----------------------------------------
    ----------------------------------------
    
    if iteration / steps_per_epoch > maxEpoch then
        break
   end
end

TEST – epoch 0.00, loss: 7374.7207	


TEST – epoch 1.07, loss: 1125.9858	


TEST – epoch 2.14, loss: 836.8282	


TEST – epoch 3.20, loss: 740.7912	


TEST – epoch 4.27, loss: 631.6577	


...

TEST – epoch 17.07, loss: 365.9571	


TEST – epoch 18.14, loss: 355.1981	


TEST – epoch 19.20, loss: 347.5113	


TEST – epoch 20.00, loss: 350.0747	


TEST – epoch 20.00, loss: 347.5504	


Let's see how our trained autoencoder reconstruct some images.

In [25]:
for i=1,10 do
    original = test_data.data[torch.random(test_data.data:size(1))]
    reconstruction = net:forward(original)
    itorch.image({original, reconstruction})
end

The reconstructions are much better than with the [autoencoder using only fully connected layers](autoencoder.ipynb).